# The Poisson equation with weak imposition of Dirichlet conditions


In this example, we consider the following Poisson problem

$$
\begin{align}
  - \nabla^2 u = f \quad \text{in $\Omega$}, \quad \quad 
  u = g            \quad \text{on $\partial \Omega$}.
\end{align}
$$

## The variational formulation

$$
\begin{align}
  \text{find $u \in V$ such that} \quad a(u,v) = l(v) \quad \forall v \in V,
\end{align}
$$

where 

- $V \subset H^1(\Omega)$, 
- $a(u,v) := \int_{\Omega} \nabla u \cdot \nabla v ~ d\Omega + \int_{\partial \Omega} \left( \kappa u v - u \partial_n v - v \partial_n u \right)~d\partial\Omega $,
- $l(v) := \int_{\Omega} f v ~ d\Omega + \int_{\partial\Omega} \left( \kappa g v - g \partial_n v \right) ~ d\Gamma$.


## Formal Model

In [ ]:
from sympde.expr import BilinearForm, LinearForm, integral, Norm
from sympde.expr     import find, EssentialBC
from sympde.topology import ScalarFunctionSpace, Square, element_of
from sympde.calculus import grad, dot, Dn
from sympde.core import Constant

from psydac.api.discretization import discretize

from sympy import pi, sin

kappa = Constant('kappa', is_real=True)

domain = Square()

V = ScalarFunctionSpace('V', domain)

x,y = domain.coordinates

u,v = [element_of(V, name=i) for i in ['u', 'v']]

# bilinear form
a  = BilinearForm((u,v), integral(domain, dot(grad(v), grad(u))))
an = BilinearForm((u,v), integral(domain.boundary, kappa*u*v - u*Dn(v) - v*Dn(u)))

# linear form
ue = sin(pi*x)*sin(pi*y)
g  = ue
f  = 2*pi**2*sin(pi*x)*sin(pi*y)

l  = LinearForm(v, integral(domain, f*v))
ln = LinearForm(v, integral(domain.boundary, kappa*g*v - g*Dn(v)))

# Variational problem
equation   = find(u, forall=v, lhs=a(u,v) + an(u,v), rhs=l(v) + ln(v))

## Discretization

In [ ]:
degree = [2,2]
ncells = [8,8]

In [ ]:
# Create computational domain from topological domain
domain_h = discretize(domain, ncells=ncells, comm=None)

# Create discrete spline space
Vh = discretize(V, domain_h, degree=degree)

# Discretize equation
equation_h = discretize(equation, domain_h, [Vh, Vh])

### Solving the PDE

In [ ]:
equation_h.set_solver('gmres', info=False, tol=1e-8)

In [ ]:
uh = equation_h.solve(kappa=1e3)

## Computing the error norm

### Computing the $L^2$ norm

In [ ]:
u = element_of(V, name='u')

# create the formal Norm object
l2norm = Norm(u - ue, domain, kind='l2')

# discretize the norm
l2norm_h = discretize(l2norm, domain_h, Vh)

# assemble the norm
l2_error = l2norm_h.assemble(u=uh)

# print the result
print(l2_error)